In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

qroq_api_key = os.getenv("GROQ_API_KEY")
qroq_api_key

'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'

In [11]:
# !pip install langchain_groq
## would allow the langchain  to communicate to llm models in groq

In [12]:
from langchain_openai import  ChatOpenAI
from langchain_groq import ChatGroq

In [17]:
model = ChatGroq(model = "gemma2-9b-it",groq_api_key = qroq_api_key)

In [18]:
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000028D6BF6FD10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000028D6BF71090>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [20]:
from langchain_core.messages import HumanMessage, SystemMessage
messages = [
    ## system message is us telling the bot how to behave
    SystemMessage(content = "Translate this from english to french"),
    HumanMessage(content = "Hello hOW ARE you?")
]

In [22]:
result = model.invoke(messages)

In [23]:
result

AIMessage(content='Here are a few ways to translate "Hello, how are you?" to French:\n\n* **Formal:**\n\n* Bonjour, comment allez-vous ? (pronounced: bohn-zhoor, koh-mahn ah-lay-voo)\n\n* **Informal:**\n\n* Salut, comment vas-tu ? (pronounced: sa-loo, koh-mah vas-too)\n*  Coucou, ça va ? (pronounced: koo-koo, sa vah)\n\n\nThe best option depends on the context and who you\'re talking to. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 21, 'total_tokens': 138, 'completion_time': 0.212727273, 'prompt_time': 0.000137589, 'queue_time': 0.013514199999999999, 'total_time': 0.212864862}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8928324b-e6fa-4518-963f-8cc445946576-0', usage_metadata={'input_tokens': 21, 'output_tokens': 117, 'total_tokens': 138})

In [25]:
## result contains the output generated by the model
## pass this result to output formatter to get the correctly formatted result
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'Here are a few ways to translate "Hello, how are you?" to French:\n\n* **Formal:**\n\n* Bonjour, comment allez-vous ? (pronounced: bohn-zhoor, koh-mahn ah-lay-voo)\n\n* **Informal:**\n\n* Salut, comment vas-tu ? (pronounced: sa-loo, koh-mah vas-too)\n*  Coucou, ça va ? (pronounced: koo-koo, sa vah)\n\n\nThe best option depends on the context and who you\'re talking to. \n'

In [26]:
## using lecl to chain the model and parser
chain = model|parser
## now when we do invoke on chain firstly mmodel called and then output of model given to parser
chain.invoke(messages)

"Hello, how are you? in French is: \n\n**Salut, comment vas-tu ?** \n\n(Informal, used with friends or people your age)\n\nor\n\n**Bonjour, comment allez-vous ?**\n\n(Formal, used with strangers or people you respect) \n\n\nLet me know if you have any other phrases you'd like translated! 😊\n"

In [28]:
## efficient way using prompt template 

## prompt templates 
from langchain_core.prompts import ChatPromptTemplate

generic_template = "convert the sentence into following language {language}"

prompt = ChatPromptTemplate(
    [("system",generic_template),
     ("user", "{text}")
     
    ]
)

In [29]:
prompt

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='convert the sentence into following language {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [32]:
result = prompt.invoke({"language" : "japanese","text":"thank you"})
## here prompt is generated for the given sentence and the laguage suggested

In [33]:
result.to_messages()

[SystemMessage(content='convert the sentence into following language japanese', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thank you', additional_kwargs={}, response_metadata={})]

In [40]:
chain = prompt|model|parser

chain.invoke({"language": "japanese","text" : "life is a prison"} )

'日本語にすると、\n\n**人生は監獄である。** \n\n(Jinsei wa kangoku de aru.)\n\n\nThis translates directly to "Life is a prison." \n'